In [1]:
%load_ext autotime
import pandas as pd
import os
import geopandas as gpd
from glob import glob
from shapely.validation import make_valid
from tqdm.auto import tqdm
import time
pd.set_option("display.max_colwidth", 100)

In [2]:
files = pd.DataFrame({"filepath": glob("Asset Directory/**", recursive=True)})
files = files[files["filepath"].str.endswith((".shp", ".zip", ".gpkg", ".geojson"))]
files["filename"] = files["filepath"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
files["filesize_GB"] = files["filepath"].apply(lambda x: os.path.getsize(x) / 1e9)
files.sort_values("filesize_GB", ascending=False).reset_index(drop=True)

,filepath,filename,filesize_GB
0,Asset Directory/LUCAS NZ Land Use/lucas-nz-land-use-map-1990-2008-2012-2016-v011.shp,lucas-nz-land-use-map-1990-2008-2012-2016-v011,1.435426
1,Asset Directory/Koordinate/smap-soil-drainage-dec-2023/smap-soil-drainage-dec-2023.shp,smap-soil-drainage-dec-2023,0.699233
2,Asset Directory/lds-new-zealand-17layers-SHP/nz-building-outlines/nz-building-outlines.shp,nz-building-outlines,0.636510
3,Asset Directory/lris-fsl-soil-drainage-class-SHP/fsl-soil-drainage-class.shp,fsl-soil-drainage-class,0.164699
4,Asset Directory/Koordinate/nzlri-land-use-capability-2021/nzlri-land-use-capability-2021.shp,nzlri-land-use-capability-2021,0.164578
5,Asset Directory/DOC/DOC_Public_Conservation_Land.shp,DOC_Public_Conservation_Land,0.139802
6,Asset Directory/Koordinate/protected-areas/protected-areas.shp,protected-areas,0.134675
7,Asset Directory/NationalRoadCentrelineDataset/roads.shp,roads,0.130881
8,Asset Directory/lds-new-zealand-17layers-SHP/nz-road-centrelines-topo-150k.shp,nz-road-centrelines-topo-150k,0.116422
9,Asset Directory/lds-new-zealand-17layers-SHP/nz-track-centrelines-topo-150k/nz-track-centrelines...,nz-track-centrelines-topo-150k,0.074558


In [3]:
polys = []
for f in tqdm(glob("Projections/*_polygon.shp")):
  df = gpd.read_file(f)
  df.geometry = df.geometry.apply(make_valid)
  polys.append(df.union_all())

  0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
if os.path.isfile("affected_assets.gpkg"):
  os.unlink("affected_assets.gpkg")

for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = pd.concat(gpd.clip(asset, poly) for poly in tqdm(polys))
  print(f"Time to clip: {time.time() - start}")
  if len(clipped_asset) == 0:
    print(f"No intersection for {row.filename}")
    continue
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')
  os.makedirs("affected_assets_CSVs", exist_ok=True)
  clipped_asset.to_csv(f"affected_assets_CSVs/{row.filename}.csv")

  0%|          | 0/42 [00:00<?, ?it/s]

Now processing wtp
Time to read file: 0.012475967407226562


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 0.05653262138366699
No intersection for wtp
Now processing wwtp
Time to read file: 0.014243841171264648


  0%|          | 0/6 [00:00<?, ?it/s]

...

Time to clip: 2.564138412475586
Now processing smap-soil-drainage-dec-2023
Time to read file: 20.64946985244751


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 15.740724086761475
Now processing nzlri-land-use-capability-2021
Time to read file: 4.3067307472229


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 12.018612623214722


In [5]:
gpd.list_layers("affected_assets.gpkg")

,name,geometry_type
0,fsl-soil-drainage-class,MultiPolygon
1,lucas-nz-land-use-map-1990-2008-2012-2016-v011,MultiPolygon
2,nz-railway-centre-lines,MultiLineString
3,nz-monument-points-topo-150k,Point
4,nz-track-centrelines-topo-150k,MultiLineString
5,linz-managed-crown-property,MultiPolygon
6,nz-drain-centrelines-topo-150k,MultiLineString
7,nz-facilities,Polygon
8,nz-road-centrelines-topo-150k,MultiLineString
9,nz-powerline-centrelines-topo-150k,MultiLineString
